In [1]:

import datetime as dt
import pyomo.environ as pyo
import pandas as pd
import os.path

In [95]:
# Datum entry
datum="2021 11 24" # provide year, month, day as a script with spaces or select from the calendar
datum = dt.datetime.strptime(datum, '%Y %m %d').date()
today = dt.date.today()


if datum<today:
    print("You selected a day in the past.")


In [96]:
## inputs we can create a table or sth similar

# department ids
deps=['CH1', 'CH2', 'CH3', 'HT', 'IM1', 'IM2', 'OR1', 'OR2']

# Departments capacities
deps_cap=[660, 435, 600,260,0, 825, 930,0]

dep_df=pd.DataFrame(zip(deps,deps_cap), columns=['dep_name','dep_cap'])

# operation room ids
op_rooms=["HLK1","HLK2","OP1","OP2","OP3","OP4","OP5","OP6"]


# Operation rooms capacities
op_rooms_cap=[435, 390, 555, 555 ,555,435,435,320]

op_df=pd.DataFrame(zip(op_rooms,op_rooms_cap), columns=["op_room_name","op_room_cap"])



In [97]:
# output show capacities of opertions rooms
op_df

,op_room_name,op_room_cap
0,HLK1,435
1,HLK2,390
2,OP1,555
3,OP2,555
4,OP3,555
5,OP4,435
6,OP5,435
7,OP6,320


In [98]:
# output show capacities of departments
dep_df

,dep_name,dep_cap
0,CH1,660
1,CH2,435
2,CH3,600
3,HT,260
4,IM1,0
5,IM2,825
6,OR1,930
7,OR2,0


In [341]:
# create a csv file to record the entries , if the file does not exist.
if os.path.isfile(f'{datum}.csv'):   
    print("it is there")
    basket=pd.read_csv(f'{datum}.csv', index_col="operation_id")
else:
    # we create an empty dataframe only first time
    basket=pd.DataFrame(columns=['dep_id','ops','weight','value',"op_room_id"])
    basket.index.name = 'operation_id'

#output

# show last 5 entries (output)
print("total weight of basket:", sum(basket['weight']))
basket.tail()

it is there
total weight of basket: 1070.0


,dep_id,ops,weight,value,op_room_id
operation_id,,,,,
xx_008,5.0,xxxx,160.0,3.0,OP4
xx_009,6.0,xxxx,90.0,2.0,OP5
xx_010,6.0,xxxx,110.0,2.0,OP5
xx_011,6.0,xxxx,145.0,2.0,OP5
xx_012,6.0,xxxx,245.0,2.0,OP5


In [342]:
# We populate basket with 5 cells: index, Abteilung-ID, ops, value, weight
# Every entry will be recorded in this table. 


ind='xx_015' # this id can be provided by the departments
basket.loc[ind,'ops']=str("xxxx") # codes of the operations to be done
basket.loc[ind,'dep_id']=3 #  Department id  who is asking for the operation
basket.loc[ind,'value']=2 # importance or the urgency of the operation
basket.loc[ind, 'weight']=160 # in minutes


basket.tail()

,dep_id,ops,weight,value,op_room_id
operation_id,,,,,
xx_009,6.0,xxxx,90.0,2.0,OP5
xx_010,6.0,xxxx,110.0,2.0,OP5
xx_011,6.0,xxxx,145.0,2.0,OP5
xx_012,6.0,xxxx,245.0,2.0,OP5
xx_015,3.0,xxxx,160.0,2.0,NaN


In [343]:
# multi-knapsack, integer divisible

mdl = pyo.ConcreteModel()

# sets
mdl.invs = pyo.Set(initialize=list(zip(basket.index, basket["dep_id"])))
mdl.bins = pyo.Set(initialize=list(op_df.index))
mdl.deps = pyo.Set(initialize=list(dep_df.index))

# params
mdl.value   = pyo.Param(mdl.invs, initialize= {(i,row["dep_id"]):row["value"] for i,row in basket.iterrows()}, mutable=True)
mdl.weight  = pyo.Param(mdl.invs, initialize= {(i,row["dep_id"]):row["weight"] for i,row in basket.iterrows()}, mutable=True)
mdl.bin_cap = pyo.Param(mdl.bins, initialize= {i:row["op_room_cap"] for i,row in op_df.iterrows()},mutable=True )
mdl.dep_cap = pyo.Param(mdl.deps, initialize= {i:row["dep_cap"] for i,row in dep_df.iterrows()}, mutable=True)



# vars
mdl.X = pyo.Var(mdl.invs, mdl.bins, within=pyo.Binary)     # the amount from invoice i in bin j



### Objective ###

mdl.OBJ = pyo.Objective(expr=sum(mdl.X[i, b]*mdl.value[i] for 
                        i in mdl.invs for
                        b in mdl.bins), sense=pyo.maximize)



### constraints ###

# don't overstuff bin
def bin_limit(self, b):
    return sum(mdl.X[i, b]*mdl.weight[i] for i in mdl.invs) <= mdl.bin_cap[b]
mdl.bin_limit = pyo.Constraint(mdl.bins, rule=bin_limit)

# one_item can be only in a single op room.
def one_item(self, i,d):
    return sum(mdl.X[i,d,b] for b in mdl.bins) <=1
mdl.one_item = pyo.Constraint(mdl.invs, rule=one_item)



# department limits

mdl.dep_limits=pyo.ConstraintList()

for d in mdl.deps:
    d_list=[]
    for i in mdl.X:
        if d==i[1]:
            d_list.append(i)    
    mdl.dep_limits.add(expr=(sum(mdl.X[i]*mdl.weight[i[:2]] for i in d_list)<=mdl.dep_cap[d])) 
        
    



# solve it...
solver = pyo.SolverFactory('glpk')
results = solver.solve(mdl)

# save the output into dataframe

count=0
for i in mdl.X:
    if pyo.value(mdl.X[i])==1:
        count+=pyo.value(mdl.X[i])
        #print(i)
        basket.loc[i[0],"op_room_id"]=op_df.loc[i[1],"op_room_name"]
        
# make sure that last item does not cause exclusion of any existing items
# If so, remove last record from the basket        
if basket.shape[0]>count:
    print('Sorry,',basket.tail(1).index,', we cannot offer you a time slot on that day.')
    basket.drop(basket.tail(1).index, inplace=True)
else:
    print(basket.tail())
    basket.to_csv(f'{datum}.csv', index=True)

              dep_id   ops  weight  value op_room_id
operation_id                                        
xx_009           6.0  xxxx    90.0    2.0        OP5
xx_010           6.0  xxxx   110.0    2.0        OP5
xx_011           6.0  xxxx   145.0    2.0        OP5
xx_012           6.0  xxxx   245.0    2.0        OP5
xx_015           3.0  xxxx   160.0    2.0        OP2


In [344]:
# print output of solution
mdl.display()

Model unknown

  Variables:
    X : Size=72, Index=X_index
        Key                : Lower : Value : Upper : Fixed : Stale : Domain
        ('xx_005', 1.0, 0) :     0 :   1.0 :     1 : False : False : Binary
        ('xx_005', 1.0, 1) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_005', 1.0, 2) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_005', 1.0, 3) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_005', 1.0, 4) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_005', 1.0, 5) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_005', 1.0, 6) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_005', 1.0, 7) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_006', 3.0, 0) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_006', 3.0, 1) :     0 :   1.0 :     1 : False : False : Binary
        ('xx_006', 3.0, 2) :     0 :   0.0 :     1 : False : False : Binary
        ('xx_006', 3.0, 3) : 